In [3]:
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
def parser(x):
    return datetime.strptime(x , '%d/%m/%Y')
ColumnList=[]

series= read_csv('DP26-skus monthly sales (FILTERED2).csv', parse_dates=['Date'] , index_col='Date', date_parser=parser)
for columns in series:
    ColumnList.append(columns)
print (ColumnList)
print(series.head())
series.plot()
pyplot.show()

c:\users\admin\appdata\local\programs\python\python36-32\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


['1000403940202', "'1000403940203", "'1000403940206", "'1000600164107", "'1000600164110", "'1001600066003", "'1001600066005", "'10020000611502", "'1002000063302", "'1002000063305", "'1002000063306", "'1002000063307", "'1002003933701", "'1002003933702", "'1002003933704", "'1002003933705", "'10020044127401", "'10020044127402", "'10020044127403", "'1004200045501", "'1004200045503", "'1004400337902", "'1004400337905", "'1004400337906", "'1005200524703", "'1005200524705", "'1005200524708", "'1005204362301", "'1005204362302", "'1005204362303", "'1005204362304", "'1011604219701", "'1011604219703", "'1011604219704", "'1011604219706", "'1011604229801", "'1011604229802", "'1011604239901", "'2000100012001", "'2000100012006", "'2000100012009", "'2000100012010", "'2000100142206", "'2000100142210", "'2000100321306", "'2001100126401", "'2001100126405", "'2001100126406", "'2001100126407", "'2001100126408", "'2001100126409", "'2001100126410", "'2001100136901", "'2001100136903", "'2001100136904", "'2001

<Figure size 640x480 with 1 Axes>

In [4]:
model={}
ColName=[]
movingavg_mape=[]
movingavg_conf=[]
movingavg_sum_error=[]
movingavg_forecast_sum=[]
model['movingavg_mape']=movingavg_mape
model['movingavg_conf']=movingavg_conf
model['movingavg_absolute_error']=movingavg_sum_error
model['movingavg forecast sum']=movingavg_forecast_sum
model['ColName']=ColName

In [5]:
model2={}
movingavg_forecast=[]
SKU=[]
model2['movingavg_forecast']=movingavg_forecast
model2['SKU']=SKU

In [6]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (((np.sum(np.abs(y_true - y_pred))) / (np.sum(y_pred))) * 100)

In [7]:
def movingavg(series,l,u):
    min = 999999999999999
    for w in range (l,u+1):
        result=[]
        for j in range(w):
            result.append(series[j])
        for i in range (w,len(series)):
            result.append((sum(series[i-w:i]))/w)
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minW=w
            minr=result
        print ('Window-size {} - mape {}'.format(w,mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('Min mape is {} - window size {} - absolute error {} - forecast sum {}'.format(mape,minW,absolute_error,forecast_sum))
    movingavg_mape.append(min)
    movingavg_conf.append(minW)
    movingavg_sum_error.append(absolute_error)
    movingavg_forecast_sum.append(forecast_sum)
    movingavg_forecast.extend(minr)
    return minr
def exponential_smoothing(series, alphaL,alphaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        result = [series[0]] # first value is same as series
        for n in range(1, (len(series))):
            result.append((alpha/10.0) * series[n-1] + (1 - (alpha/10.0)) * result[n-1])
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minalpha=(alpha/10.0)
            minr=result
        print ('alpha {} - mape {}'.format((alpha/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('best alpha {} - mape is {} - absolute error {} - forecast sum {}'.format(minalpha,min,absolute_error,forecast_sum))
    simpleExpo_mape.append(min)
    simpleExpo_conf.append(minalpha)
    simpleExpo_sum_error.append(absolute_error)
    simpleExpo_forecast_sum.append(forecast_sum)
    simpleExpo_forecast.extend(minr)
    return minr

def double_exponential_smoothing(series, alphaL, alphaU, betaL, betaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        for beta in range (betal, betau+1):
            result = [series[0], series[0]]
            for n in range(2, len(series)):
                if n == 2:
                    level, trend = series[0], series[1] - series[0]
                if n <= len(series): # we are forecasting
                    value = series[n-1]
                last_level, level = level, (alpha/10.0)*value + (1-(alpha/10.0))*(level+trend)
                trend = (beta/10.0)*(level-last_level) + (1-(beta/10.0))*trend
                result.append(level+trend)
            mape=mean_absolute_percentage_error(series[24:],result[24:])
            #print result
            if (0<=mape<min):
                min=mape
                minalpha=(alpha/10.0)
                minbeta=(beta/10.0)
                minr=result
            print ('alpha {} - beta {} - mape {}'.format((alpha/10.0),(beta/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('min alpha {} - min beta {} - mape {} - absolute error {} - forecast sum {}'.format(minalpha,minbeta,min,absolute_error,forecast_sum))
    DoubleExpo_mape.append(min)
    DoubleExpo_conf.append([minalpha,minbeta])
    DoubleExpo_sum_error.append(absolute_error)
    DoubleExpo_forecast_sum.append(forecast_sum)
    DoubleExpo_forecast.extend(minr)
    return minr

def M_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def M_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def M_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = M_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = M_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) * seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val/seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val/smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append((smooth+trend)*seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    M_tripleExpo_mape.append(min2)
    M_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    M_tripleExpo_sum_error.append(absolute_error)
    M_tripleExpo_forecast_sum.append(forecast_sum)
    M_tripleExpo_forecast.extend(bestr)
    return bestr

def A_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def A_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def A_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = A_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = A_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) + seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val-seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val-smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append(smooth+trend+seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    A_tripleExpo_mape.append(min2)
    A_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    A_tripleExpo_sum_error.append(absolute_error)
    A_tripleExpo_forecast_sum.append(forecast_sum)
    A_tripleExpo_forecast.extend(bestr)
    return bestr

In [8]:
def forecasting ():
    for i in range(len(ColumnList)):
        ColName.append(ColumnList[i])
        for ik in range(0,36):
            SKU.append(ColumnList[i])
        print (ColumnList[i])
        print (i)
        print ("")
        movingavg(series[ColumnList[i]],2,12)

In [9]:
forecasting()

1000403940202
0

Window-size 2 - mape 44.87361800210963
Window-size 3 - mape 41.000265557227586
Window-size 4 - mape 42.99251707956822
Window-size 5 - mape 43.48298751917069
Window-size 6 - mape 41.98743175468934
Window-size 7 - mape 42.268236207645316
Window-size 8 - mape 40.19878860518376
Window-size 9 - mape 40.445627652545554
Window-size 10 - mape 40.316749259038154
Window-size 11 - mape 38.31040865066261
Window-size 12 - mape 37.51144327167523
Min mape is 37.51144327167523 - window size 12 - absolute error 14111.94 - forecast sum 37620.36
'1000403940203
1

Window-size 2 - mape 25.71176075813152
Window-size 3 - mape 23.449527895997583
Window-size 4 - mape 21.19852320550581
Window-size 5 - mape 21.799631212106743
Window-size 6 - mape 22.093733382909736
Window-size 7 - mape 22.5228008275824
Window-size 8 - mape 20.721822746032423
Window-size 9 - mape 20.888975039412166
Window-size 10 - mape 21.346071681437532
Window-size 11 - mape 20.542686495655037
Window-size 12 - mape 20.516547872

Window-size 2 - mape 40.19050594814158
Window-size 3 - mape 34.400204955090736
Window-size 4 - mape 38.02267731366471
Window-size 5 - mape 38.430046186428086
Window-size 6 - mape 39.31702560501015
Window-size 7 - mape 40.70909250337466
Window-size 8 - mape 40.5935336288066
Window-size 9 - mape 37.330836028678526
Window-size 10 - mape 34.738664586501265
Window-size 11 - mape 39.39596839862507
Window-size 12 - mape 46.47443820366427
Min mape is 46.47443820366427 - window size 3 - absolute error 4565.320000000001 - forecast sum 13271.199999999997
'10020044127401
16

Window-size 2 - mape 40.21244309559939
Window-size 3 - mape 38.84462151394422
Window-size 4 - mape 39.926199261992615
Window-size 5 - mape 40.09378663540445
Window-size 6 - mape 38.36715431807456
Window-size 7 - mape 37.996820349761535
Window-size 8 - mape 43.38057379882475
Window-size 9 - mape 43.11414392059553
Window-size 10 - mape 47.23717767072825
Window-size 11 - mape 50.026441036488634
Window-size 12 - mape 52.2687390323

Window-size 12 - mape 58.734242346282485
Min mape is 58.734242346282485 - window size 2 - absolute error 915.0 - forecast sum 1977.0
'1011604219701
31

Window-size 2 - mape 32.301726642671206
Window-size 3 - mape 29.26195298335456
Window-size 4 - mape 28.935329802950367
Window-size 5 - mape 29.53137981687818
Window-size 6 - mape 30.45436957286352
Window-size 7 - mape 29.87735039783903
Window-size 8 - mape 29.37029062451957
Window-size 9 - mape 29.681447004552464
Window-size 10 - mape 29.96168002515142
Window-size 11 - mape 29.25511978895156
Window-size 12 - mape 29.550832500465297
Min mape is 29.550832500465297 - window size 4 - absolute error 262356.77999999997 - forecast sum 906700.5
'1011604219703
32

Window-size 2 - mape 16.576200958176873
Window-size 3 - mape 15.556050003073457
Window-size 4 - mape 13.507569996184499
Window-size 5 - mape 12.90471332370545
Window-size 6 - mape 13.669800765020684
Window-size 7 - mape 12.945640004055225
Window-size 8 - mape 11.096404175713035
Window-

Window-size 2 - mape 75.25728049047514
Window-size 3 - mape 65.34323862842413
Window-size 4 - mape 66.25062535288619
Window-size 5 - mape 62.11319887599386
Window-size 6 - mape 59.722967764021185
Window-size 7 - mape 60.23786580855809
Window-size 8 - mape 63.946067024132766
Window-size 9 - mape 65.58389346845105
Window-size 10 - mape 67.28400770178882
Window-size 11 - mape 66.44114804278622
Window-size 12 - mape 63.122039124374496
Min mape is 63.122039124374496 - window size 6 - absolute error 39170000.0 - forecast sum 65586158.0
'2001100136903
53

Window-size 2 - mape 47.75476497158589
Window-size 3 - mape 40.81388416392928
Window-size 4 - mape 45.08364055700523
Window-size 5 - mape 48.168448174540856
Window-size 6 - mape 50.8091151909465
Window-size 7 - mape 52.03945162285226
Window-size 8 - mape 53.61492544170462
Window-size 9 - mape 54.02428421941852
Window-size 10 - mape 52.69067214904204
Window-size 11 - mape 51.357495822598665
Window-size 12 - mape 48.147038787143494
Min mape is

Window-size 7 - mape 141.84473063418918
Window-size 8 - mape 141.69634185820655
Window-size 9 - mape 136.09681235412603
Window-size 10 - mape 130.68666773785904
Window-size 11 - mape 122.19989249944565
Window-size 12 - mape 114.06038255300777
Min mape is 114.06038255300777 - window size 2 - absolute error 206284.08000000002 - forecast sum 250128.00000000003
'20011039511001
70

Window-size 2 - mape 171.42857142857142
Window-size 3 - mape 160.0
Window-size 4 - mape 147.36842105263156
Window-size 5 - mape 149.01960784313727
Window-size 6 - mape 118.75
Window-size 7 - mape 118.5185185185185
Window-size 8 - mape 113.33333333333333
Window-size 9 - mape 114.84374999999997
Window-size 10 - mape 109.27152317880793
Window-size 11 - mape 101.7605633802817
Window-size 12 - mape 98.08153477218225
Min mape is 98.08153477218225 - window size 12 - absolute error 104704.0 - forecast sum 106752.0
'20011039511002
71

Window-size 2 - mape 56.510007412898446
Window-size 3 - mape 39.65296803652967
Window-si

...

Window-size 6 - mape 139.36048594040557
Window-size 7 - mape 146.37263136821397
Window-size 8 - mape 152.2732577159514
Window-size 9 - mape 152.882930956869
Window-size 10 - mape 148.246398119158
Window-size 11 - mape 143.13247469063833
Window-size 12 - mape 136.73980508520717
Min mape is 136.73980508520717 - window size 2 - absolute error 17295.84 - forecast sum 19104.120000000003
'80064018335901
134

Window-size 2 - mape 110.9375
Window-size 3 - mape 122.74305555555556
Window-size 4 - mape 132.81249999999997
Window-size 5 - mape 132.11009174311926
Window-size 6 - mape 132.8125
Window-size 7 - mape 136.39788997739262
Window-size 8 - mape 132.75008253549026
Window-size 9 - mape 129.90758891066926
Window-size 10 - mape 127.34055354993983
Window-size 11 - mape 120.97488921713442
Window-size 12 - mape 114.46552695848207
Min mape is 114.46552695848207 - window size 2 - absolute error 51.12 - forecast sum 46.08
'80064018517601
135

Window-size 2 - mape 200.8
Window-size 3 - mape 182.5327510


Window-size 2 - mape 56.55588376706668
Window-size 3 - mape 56.94830390131599
Window-size 4 - mape 52.4297710467159
Window-size 5 - mape 48.97689472812047
Window-size 6 - mape 44.43825704537046
Window-size 7 - mape 43.6013323008875
Window-size 8 - mape 41.82819302666709
Window-size 9 - mape 43.37542295713222
Window-size 10 - mape 42.40767852800187
Window-size 11 - mape 41.771343738735496
Window-size 12 - mape 40.54461772675276
Min mape is 40.54461772675276 - window size 12 - absolute error 240387.81 - forecast sum 592896.97
'80072021611203
149

Window-size 2 - mape 42.550033134526174
Window-size 3 - mape 41.24667551715411
Window-size 4 - mape 38.03257062728861
Window-size 5 - mape 33.99815322931482
Window-size 6 - mape 30.72673316824516
Window-size 7 - mape 30.0629137936949
Window-size 8 - mape 30.612327704516968
Window-size 9 - mape 31.582988995885557
Window-size 10 - mape 30.8423923264999
Window-size 11 - mape 30.28414708757323
Window-size 12 - mape 30.44462165627851
Min mape is 30.

Window-size 2 - mape 82.92011019283747
Window-size 3 - mape 70.90255620346937
Window-size 4 - mape 61.027931728568674
Window-size 5 - mape 66.18124094140381
Window-size 6 - mape 64.80231819625277
Window-size 7 - mape 62.91780533949969
Window-size 8 - mape 65.49201704726161
Window-size 9 - mape 67.33835640441563
Window-size 10 - mape 68.45742670482156
Window-size 11 - mape 69.21088298780184
Window-size 12 - mape 70.7580629906365
Min mape is 70.7580629906365 - window size 4 - absolute error 663.99 - forecast sum 1088.01
'80082026719601
166

Window-size 2 - mape 37.42711370262391
Window-size 3 - mape 32.736855393549206
Window-size 4 - mape 39.04687954964573
Window-size 5 - mape 40.470292084548696
Window-size 6 - mape 41.157374976507384
Window-size 7 - mape 45.23926034669395
Window-size 8 - mape 44.420102756696146
Window-size 9 - mape 44.29857984066506
Window-size 10 - mape 43.622857497081355
Window-size 11 - mape 42.41816023067791
Window-size 12 - mape 41.58820241626897
Min mape is 41.588

Window-size 11 - mape 67.07145547195042
Window-size 12 - mape 63.53012706231516
Min mape is 63.53012706231516 - window size 12 - absolute error 1849.4700000000003 - forecast sum 2911.17
'80082029316501
182

Window-size 2 - mape 42.75458447132267
Window-size 3 - mape 39.74053409891121
Window-size 4 - mape 45.66639325299179
Window-size 5 - mape 43.028793930379855
Window-size 6 - mape 45.73328510614619
Window-size 7 - mape 47.54054243792854
Window-size 8 - mape 47.47695390781564
Window-size 9 - mape 48.81197528908602
Window-size 10 - mape 48.74110197821214
Window-size 11 - mape 49.05538962991977
Window-size 12 - mape 49.197095093851786
Min mape is 49.197095093851786 - window size 3 - absolute error 586.92 - forecast sum 1476.88
'8008202982701
183

Window-size 2 - mape 108.7696780379179
Window-size 3 - mape 97.49386476461615
Window-size 4 - mape 102.04468063553414
Window-size 5 - mape 107.00157629573862
Window-size 6 - mape 110.39660201013761
Window-size 7 - mape 110.08386794262455
Window-

Window-size 6 - mape 40.316085161588234
Window-size 7 - mape 41.482995417801085
Window-size 8 - mape 39.835153854882066
Window-size 9 - mape 38.11741745373519
Window-size 10 - mape 38.34831176533805
Window-size 11 - mape 38.241633043685475
Window-size 12 - mape 37.12884028988859
Min mape is 37.12884028988859 - window size 12 - absolute error 21219.73 - forecast sum 57151.61
'80124039211403
203

Window-size 2 - mape 43.24875961113756
Window-size 3 - mape 42.95302519967106
Window-size 4 - mape 42.73310145342775
Window-size 5 - mape 37.730627213419
Window-size 6 - mape 35.548214444178086
Window-size 7 - mape 34.1679044000671
Window-size 8 - mape 32.164387120105644
Window-size 9 - mape 32.587634344898916
Window-size 10 - mape 30.870930835916084
Window-size 11 - mape 29.53745263306634
Window-size 12 - mape 29.37875320864788
Min mape is 29.37875320864788 - window size 12 - absolute error 98818.53000000001 - forecast sum 336360.53


In [ ]:
print (model2)
df=pd.DataFrame(model2)
print (df)

In [ ]:
print (model)
df1=pd.DataFrame(model)
print (df1)